In [ ]:
import rtsvg
from shapely.geometry import Point, Polygon
import random
rt = rtsvg.RACETrack()

In [ ]:
def pyramidMethodDiagram(level_lu, description=None, level_min_h=32, point_r=4.0, txt_h=12, w=256, h=240, x_ins=12, y_ins=12):
    _svg_ = [f'<svg x="0" y="0" width="{w}" height="{h}">']
    _svg_.append(f'<rect x="0" y="0" width="{w}" height="{h}" fill="white"/>')
    # Three points of triangle
    xyt, tri_w = (w/2.0, y_ins), w - 2*x_ins
    if description is not None: y_base, tri_h = h - y_ins - txt_h, h - txt_h
    else:                       y_base, tri_h = h - y_ins, h
    xyb1, xyb2 = (x_ins, y_base), (w-x_ins, y_base)
    tri_area = tri_w * tri_h / 2.0

    # Inner triangle
    xyt_i = (xyt[0], xyt[1] + y_ins)
    xyb1_i, xyb2_i = (xyb1[0] + x_ins, xyb1[1] - y_ins/2.0), (xyb2[0] - x_ins, xyb2[1] - y_ins/2.0)

    # Point In Inner Triangle
    def pointInInnerTriangle(x, y):
        return Point(x, y).within(Polygon([xyt_i, xyb1_i, xyb2_i]))

    # Return The Two X Coordinates At This Y
    def triangleXatY(y):
        horz     = ((0,y),(w,y))
        b1_tuple = rt.segmentsIntersect(horz, (xyt, xyb1))
        b2_tuple = rt.segmentsIntersect(horz, (xyt, xyb2))
        return b1_tuple[1], b2_tuple[1]

    # Fill in dots between these two points
    def dotFill(n, y0, y1):
        _multiple_ = 4.0
        _misses_   = 0
        xys        = set()
        for i in range(n):
            point_too_close = True
            while point_too_close:
                point_too_close         = False
                point_in_inner_triangle = False
                while point_in_inner_triangle == False:
                    x, y = x_ins + random.random() * tri_w, y0 + random.random() * (y1 - y0)
                    point_in_inner_triangle = pointInInnerTriangle(x, y)
                for _xy_ in xys:
                    l = rt.segmentLength((_xy_, (x, y)))
                    if l < _multiple_*point_r: 
                        point_too_close  = True
                        _misses_        += 1
                        if _misses_ > 50:
                            _multiple_ -= 0.5
                            if _multiple_ < 0.5: _multiple_ = 0.5
                            _misses_    = 0
                        break
            xys.add((x, y))
            _svg_.append(f'<circle cx="{x}" cy="{y}" r="{point_r}" fill="#000000"/>')

    l_bottom, l_top = min(level_lu.keys()), max(level_lu.keys())
    levels          = l_top - l_bottom + 1

    # fill in any empty levels
    for _level_ in range(l_bottom, l_top+1):
        if _level_ not in level_lu.keys(): level_lu[_level_] = 0

    if levels > 1:
        levels_h, levels_w_data = {}, 0
        for _level_ in range(l_bottom, l_top+1):
            if _level_ not in level_lu.keys() or level_lu[_level_] == 0: levels_h[_level_] = level_min_h
            else:                                                        levels_w_data     = levels_w_data + 1
        h_left_over = tri_h - (levels - levels_w_data)
        h_per_level = h_left_over / levels_w_data
        for _level_ in range(l_bottom, l_top+1):
            if _level_ not in levels_h: levels_h[_level_] = h_per_level
        # Now, draw the levels
        y_base = xyb1[1]
        for _level_ in range(l_bottom, l_top+1):
            y_other = y_base - levels_h[_level_]
            x0, x1  = triangleXatY(y_base)
            _svg_.append(f'<line x1="{x0}" y1="{y_base}" x2="{x1}" y2="{y_base}" stroke="#000000" stroke-width="2.0"/>')
            _svg_.append(rt.svgText(str(_level_), w/2.0, y_base-4, txt_h=2.5*txt_h, color='#d0d0d0', anchor='middle'))
            dotFill(level_lu[_level_], y_other+y_ins, y_base-y_ins)
            y_base -= levels_h[_level_]
    else:
        # Fill in the only level
        dotFill(level_lu[l_bottom], xyt_i[1], xyb1_i[1])

    # Outline of triangle
    triangle_path = f'M {xyt[0]} {xyt[1]} L {xyb1[0]} {xyb1[1]} L {xyb2[0]} {xyb2[1]} Z'
    _svg_.append(f'<path d="{triangle_path}" fill="none" stroke="#000000" stroke-width="2.0"/>')

    # Inner triangle
    #triangle_path = f'M {xyt_i[0]} {xyt_i[1]} L {xyb1_i[0]} {xyb1_i[1]} L {xyb2_i[0]} {xyb2_i[1]} Z'
    #_svg_.append(f'<path d="{triangle_path}" fill="none" stroke="#000000" stroke-width="1.0"/>')

    # Description text (if set)
    if description is not None: _svg_.append(rt.svgText(description, w/2.0, h-y_ins/2.0, txt_h=txt_h, anchor='middle'))
    _svg_.append('</svg>')
    return ''.join(_svg_)

_level_lu_  = {1:10,  2:5,  3:3}        # level 0 is the base, 1 is the first level, 2 is the second level (top level, in this case)
_level2_lu_ = {0:40, 1:10, 2:3, 3:0}    # w/ four levels
_level3_lu_ = {0:0,   3:1}              # w/ four levels... only one has data
_level4_lu_ = {5:200}                   # one level, lots of data

rt.tile([pyramidMethodDiagram(_level_lu_),
         pyramidMethodDiagram(_level_lu_, 'Test'),
         pyramidMethodDiagram(_level2_lu_),
         pyramidMethodDiagram(_level3_lu_),
         pyramidMethodDiagram(_level4_lu_)], spacer=10)
